# Annotating your Images for Object Detection

This tutorial will walk you through the process of using the Appen Data Annotation Platform to create a dataset of annotated images to train a machine learning model. At the end of this tutorial, you will be able to upload data, run your own job in ADAP, and prepare the data into the Pascal VOC 2007 format using in ML platforms like NVIDIA’s Transfer Learning Toolkit. If you have any questions about the platform, please visit our [Success Center](https://success.appen.com/).

```Author: Rahul Parundekar, Director of Data Science, Appen. ```

## 1. Installing & Importing Dependencies

In [1]:
# First we install the needed libraries for this notebook
!pip install pandas imageio tqdm
# Install unofficial v1.8 of dicttoxml based on https://github.com/quandyfactory/dicttoxml/pull/64
!pip install https://github.com/Simon-Campbell/dicttoxml/archive/b0777caf0ccf64c97a9c47a70b9ced6e0c51ddf7.zip

  Using cached https://github.com/Simon-Campbell/dicttoxml/archive/b0777caf0ccf64c97a9c47a70b9ced6e0c51ddf7.zip


In [2]:
# Next, we import the libraries in this notebook
import os
import shutil
import pandas as pd
import imageio
import json
import glob
from tqdm import tqdm
from dicttoxml import dicttoxml
from xml.dom.minidom import parseString

## 2. Preparing your data for loading into ADAP

### a. Loading your data into a web-accesible location
First, upload your images into the cloud or a location that ADAP can access. For example, you can load them into a private S3 bucket. 

### b. Prepare your CSV
Structure your input CSV file. Your CSV will have at least two columns.

1. URLs to the images for ADAP. You can do this in one of three ways:
- Use publicly-available URLs for your data from your S3 bucket.
- Use pre-signed URLS for the duration of your annotation (e.g 2 weeks).
- Leverage Appen’s Secure Data Access tool, which you can use to attach your database securely to the platform; Appen will only access your data when needed.

2. Local file name on your device/some identification mechanism that you can later use.

### c. Here's how your CSV would look like

In [3]:
# Preview the CSV
input_data_filepath = "/Users/rparundekar/Downloads/coco-animals/coco_animals.csv"
input_df = pd.read_csv(input_data_filepath)
input_df.head()

,filename,image_url
0,/Users/rparundekar/Downloads/coco-animals/trai...,https://figure-eight-datasets.s3.amazonaws.com...
1,/Users/rparundekar/Downloads/coco-animals/trai...,https://figure-eight-datasets.s3.amazonaws.com...
2,/Users/rparundekar/Downloads/coco-animals/trai...,https://figure-eight-datasets.s3.amazonaws.com...
3,/Users/rparundekar/Downloads/coco-animals/trai...,https://figure-eight-datasets.s3.amazonaws.com...
4,/Users/rparundekar/Downloads/coco-animals/trai...,https://figure-eight-datasets.s3.amazonaws.com...


## 3. Getting your Data Annotated

### a. Sign Up / Log In

To get data annotated, you'll need  to create an account at https://client.appen.com/

![Signup](signup.png)

### b. Create Your Job

Starting at the jobs tab on the left (or the "home" tab that shows you the templates below), you can select "Create a Job" to begin. 

![Create your Job - 0](create_0.png "Job Page")

You can choose from a variety of templates. In our case we will first choose "Image Annotation".

![Create your Job - 1](create_1.png "Create your Job")


For our use case, we then choose "Annotate and Categorize Objects in an Image Using a Bounding Box"

![Create your Job - 2](create_2.png "Create your Job")

### c. Upload Data 

Upload the csv you created earlier by dragging and dropping into the upload tab.

![Upload 1](upload_1.png "Upload")

You'll see the data once it's loaded.

![Upload 2](upload_2.png "Upload")

### d. Design your Job 

Provide guidelines for Appen’s Crowd of over one million data labelers on what they should be looking for and any requirements they should be aware of.

With the template, we've provided you a simple job design to get you started.

![Design 1](design_1.png "Design")

You'll need to also define the classes that should be detected by clicking "Manage the Image Annotation Ontology".

![Design 2](design_2.png "Design")

You can update the instructions to give more context about your use case and describe how the annotators should identify and label objects in the images. 

![Design 3](design_3.png "Design")

Preview the job so that you can see how the annotator will see it. 

![Preview](preview_1.png "Preview")

![Preview](preview_2.png "Preview")

### e. Quality & Settings

Create test questions to measure and track labeler performance.

![QA](qa.png "QA")

### f. Launch your Job

Do a test run first before officially launching your annotation job on the platform. Once you’ve launched your job, Appen’s global crowd of data labelers will annotate your data to your specifications. Make sure you've added enough funds to be able to launch your job!

![Launch](launch.png "Launch")

### g. Monitor

Monitor the accuracy rate of annotations in real-time. Make adjustments as needed in areas like job design, test questions, or annotators.

![Monitor](monitor.png "Monitor")

### h. Results

Download a report of your labeled data output by clicking on "Download" for the "Full" Report.

![Results](results.png "Results")

## 4. Converting to Pascal VOC Format

From here, you can use the following section to convert your labeled data to a format like the Pascal Visual Object Classes (VOC)
format.

In [4]:
# Let's see at how the downloaded data looks like.
adap_annotations_data_filepath = "/Users/rparundekar/Downloads/f1746_JOB_ID.csv"
df = pd.read_csv(adap_annotations_data_filepath)
df.head()

,_unit_id,_created_at,_id,_started_at,_tainted,_channel,_trust,_worker_id,_country,_region,_city,_ip,annotation,nothing_to_box,annotation_gold,filename,image_url,nothing_to_box_gold
0,2989031505,3/19/2021 21:38:03,5984571461,3/19/2021 21:29:41,False,feca,1.0,45227798,VEN,17,Altagracia,190.36.245.70,"[{""id"":""2931ed45-5ab3-4915-9a0b-a29d30922447"",...",NaN,NaN,/Users/rparundekar/Downloads/coco-animals/trai...,https://figure-eight-datasets.s3.amazonaws.com...,NaN
1,2989031505,3/19/2021 21:39:41,5984573163,3/19/2021 21:35:57,False,feca,1.0,45163079,EGY,01,Mansoura,156.199.241.173,"[{""id"":""b44498a1-30c0-41b2-8dfc-af648888e0a1"",...",NaN,NaN,/Users/rparundekar/Downloads/coco-animals/trai...,https://figure-eight-datasets.s3.amazonaws.com...,NaN
2,2989031505,3/19/2021 21:51:30,5984579758,3/19/2021 21:29:42,False,feca,1.0,46316811,USA,MT,Billings,131.196.55.127,"[{""id"":""1e8999d3-e6b3-44a6-8496-698122b6a7f2"",...",NaN,NaN,/Users/rparundekar/Downloads/coco-animals/trai...,https://figure-eight-datasets.s3.amazonaws.com...,NaN
3,2989031506,3/19/2021 21:36:08,5984568481,3/19/2021 21:29:42,False,feca,1.0,46017640,VEN,22,Chivacoa,190.206.15.172,"[{""id"":""7bf5220c-7ca8-494d-af14-e2622cb089ce"",...",NaN,NaN,/Users/rparundekar/Downloads/coco-animals/trai...,https://figure-eight-datasets.s3.amazonaws.com...,NaN
4,2989031506,3/19/2021 21:38:00,5984571362,3/19/2021 21:29:43,False,feca,1.0,46414011,BRA,27,Sao Paulo,191.96.70.45,"[{""id"":""36bb558c-3bde-4438-b01e-dfdbda5435ce"",...",NaN,NaN,/Users/rparundekar/Downloads/coco-animals/trai...,https://figure-eight-datasets.s3.amazonaws.com...,NaN


In [5]:
# We'll only need to look at two of the columns - the filename, 
# that ties the data with our local files, and annotation that are the object annotations.
df[["filename", "annotation"]]

,filename,annotation
0,/Users/rparundekar/Downloads/coco-animals/trai...,"[{""id"":""2931ed45-5ab3-4915-9a0b-a29d30922447"",..."
1,/Users/rparundekar/Downloads/coco-animals/trai...,"[{""id"":""b44498a1-30c0-41b2-8dfc-af648888e0a1"",..."
2,/Users/rparundekar/Downloads/coco-animals/trai...,"[{""id"":""1e8999d3-e6b3-44a6-8496-698122b6a7f2"",..."
3,/Users/rparundekar/Downloads/coco-animals/trai...,"[{""id"":""7bf5220c-7ca8-494d-af14-e2622cb089ce"",..."
4,/Users/rparundekar/Downloads/coco-animals/trai...,"[{""id"":""36bb558c-3bde-4438-b01e-dfdbda5435ce"",..."
...,...,...
295,/Users/rparundekar/Downloads/coco-animals/trai...,"[{""id"":""47a719de-b3f1-4258-b698-5268124a4e54"",..."
296,/Users/rparundekar/Downloads/coco-animals/trai...,"[{""id"":""503b0944-faf5-49bf-a5f7-03b789234226"",..."
297,/Users/rparundekar/Downloads/coco-animals/trai...,"[{""id"":""4d15a488-7311-4252-9e51-d0858a4cc095"",..."
298,/Users/rparundekar/Downloads/coco-animals/trai...,"[{""id"":""2a31ad1f-32c2-4162-9de8-ce4d6f7b4474"",..."


In [6]:
# Here's how one annotation looks like:
print(json.dumps(json.loads(df["annotation"][0]), indent=2))

[
  {
    "id": "2931ed45-5ab3-4915-9a0b-a29d30922447",
    "class": "cat",
    "type": "box",
    "coordinates": {
      "x": 210,
      "y": 32,
      "w": 364,
      "h": 394
    }
  }
]


In [7]:
# Clean up the data frame and replace empty cells in the 'annotation' column without any detected objects.
df.fillna('', inplace=True)

In [8]:
# Let's create a folder for storing the annotations
pascal_annotations_folder = "/Users/rparundekar/Downloads/coco-animals/annotations"

# Create the folder if it doesn't exist
if not os.path.exists(pascal_annotations_folder):
    os.makedirs(pascal_annotations_folder)

In [9]:
# Iterate through the data
for index, row in tqdm(df.iterrows(), total=df.shape[0]): 
    # Create the annotation object to write to file
    annotation = {} 

    # Get the path to the local file
    filename = row["filename"]
    
    # File details
    annotation["path"] = os.path.abspath(filename)
    annotation["filename"] = os.path.basename(filename)    
    annotation["folder"] = annotation["path"].split(os.path.sep)[-2]
    
    # Dataset details 
    annotation["source"] = {"dataset": "coco-animals"}
    
    # Image details 
    height, width, depth = imageio.imread(filename).shape
    annotation["size"] = {"width": width, "height": height, "depth":depth}
    
    # Annotation is bounding box
    annotation["segmented"] = 0
    
    # Check if no objects in image
    if row["nothing_to_box"]:
        pass # No objects to add
    else:
        # Objects annotated
        adap_objects = json.loads(row["annotation"])
        objects = []
        annotation["object"] = objects
        for adap_obj in adap_objects:
            # Sanity checks
            assert adap_obj["type"]=="box"
            assert "angle" not in adap_obj

            coordinates = adap_obj["coordinates"]
            x, y, w, h = coordinates["x"], coordinates["y"], coordinates["w"], coordinates["h"],
            obj = {"name":adap_obj["class"], "bndbox": {"xmin": x, "ymin": y, "xmax": (x+w), "ymax": (y+h)}}
            objects.append(obj)
            objects.append(obj)
    
    # Outermost root object
    root_annotation_obj = {"annotation": annotation}

    # Convert to XML
    item_name = lambda x: x
    xml_snippet = dicttoxml(root_annotation_obj, root=False, attr_type=False, item_func=item_name, fold_list=False).decode("utf-8")
    dom = parseString(xml_snippet)
    
    # Write to file (Note: will overwrite files w/ same name)
    split = annotation["filename"].split(".")
    split[-1] = ".xml" # Change the extension from whatever (e.g. .jpg/.jpeg/.bmp/png) to .xml
    xml_filename = ''.join(split)
    with open(os.path.join(pascal_annotations_folder, xml_filename), "w") as f:
        f.write(dom.toprettyxml())

100%|██████████| 300/300 [00:02<00:00, 118.31it/s]


## That's it! Your data is ready for training! 

Let's check one file for confirming how the XML looks like. 

In [10]:
# Print one file content
for filename in glob.glob(os.path.join(pascal_annotations_folder, "*.xml")):
    print(f"File: {filename}")
    print(f"Contents:")
    with open(os.path.join(pascal_annotations_folder, xml_filename), "r") as f:
        print(f.read())
    break

File: /Users/rparundekar/Downloads/coco-animals/annotations/COCO_train2014_000000069003.xml
Contents:
<?xml version="1.0" ?>
<annotation>
	<path>/Users/rparundekar/Downloads/coco-animals/train/cat/COCO_train2014_000000119488.jpg</path>
	<filename>COCO_train2014_000000119488.jpg</filename>
	<folder>cat</folder>
	<source>
		<dataset>coco-animals</dataset>
	</source>
	<size>
		<width>640</width>
		<height>425</height>
		<depth>3</depth>
	</size>
	<segmented>0</segmented>
	<object>
		<name>cat</name>
		<bndbox>
			<xmin>447</xmin>
			<ymin>370</ymin>
			<xmax>504</xmax>
			<ymax>424</ymax>
		</bndbox>
	</object>
	<object>
		<name>cat</name>
		<bndbox>
			<xmin>447</xmin>
			<ymin>370</ymin>
			<xmax>504</xmax>
			<ymax>424</ymax>
		</bndbox>
	</object>
</annotation>



## Training your Model

Your data annotated with Appen can now be used to train your object detection model. 

### Transfer Learning with NVIDIA's Transfer Learning Toolkit 

The annnotated data generated with ADAP can also be used with NVIDIA's Transfer Learning Toolkit (TLT). TLT is a python based AI toolkit for customizing pre-trained AI models with your own data. TLT allows you to train, fine tune, prune and export highly optimized and accurate AI models for edge deployment by adapting popular network architectures and backbones to your data.

- [Transfer Learning with Object Detection](https://ngc.nvidia.com/catalog/models/nvidia:tlt_pretrained_object_detection) with common architectures like YOLOV3, FasterRCNN, SSD, DSSD, and RetinaNet. 
- [Transfer Learning with Detectnet v2](https://ngc.nvidia.com/catalog/models/nvidia:tlt_pretrained_detectnet_v2) 